In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

with open('/content/drive/My Drive/data/train.jsonl') as f:
    train_data = pd.read_json(f, lines=True)
with open('/content/drive/My Drive/data/validation.jsonl') as f:
    test_data = pd.read_json(f, lines=True)

In [3]:
train_data.response = train_data.response.str.replace('@USER', '', regex=False)
train_data.response = train_data.response.str.replace('<URL>', '', regex=False)

test_data.response = test_data.response.str.replace('@USER', '', regex=False)
test_data.response = test_data.response.str.replace('<URL>', '', regex=False)

In [4]:
train_data.context = train_data.context.apply(lambda x: [str.replace(s, '@USER', '') for s in x])
train_data.context = train_data.context.apply(lambda x: [str.replace(s, '<URL>', '') for s in x])

test_data.context = test_data.context.apply(lambda x: [str.replace(s, '@USER', '') for s in x])
test_data.context = test_data.context.apply(lambda x: [str.replace(s, '<URL>', '') for s in x])

In [5]:
!pip install ekphrasis

     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 184kB 31.8MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp36-none-any.whl size=82844 sha256=c05e435a16ecce4296580b4b4b80bd69f2ae86e364f0cd35e5b947a5e94f6442
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=fcc29aeeb69ef2bdc589f20fdf2f20dbbdaea6c48c71fd8dbc4dbca42d01b000
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ekphrasis ftfy


In [6]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=False,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [7]:
train_data.response = train_data.response.apply(text_processor.pre_process_doc)
train_data.context = train_data.context.apply(lambda x: [text_processor.pre_process_doc(s) for s in x])

test_data.response = test_data.response.apply(text_processor.pre_process_doc)
test_data.context = test_data.context.apply(lambda x: [text_processor.pre_process_doc(s) for s in x])

In [8]:
def rev_and_join(x):
  #x.reverse() 
  #return ' '.join(x)
  return x.pop()

train_data.context = train_data.context.apply(rev_and_join)
test_data.context = test_data.context.apply(rev_and_join)

In [9]:
train_data.context

0       [If, your, child, is, not, named, Barron, ., <...
1       [having, to, make, up, excuses, of, why, your,...
2       [I, ’, ll, remember, to, not, support, you, at...
3       [But, not, half, as, stupid, as, Schiff, looks...
4       [They, already, did, ., Obama, said, many, tim...
                              ...                        
4995    [We, will, work, on, this, and, reach, out, at...
4996    [Yup, she, went, there, ., Nah, not, at, all, ...
4997    [", some, look, at, silence, as, boredom, or, ...
4998    [Lots, of, people, believed, —, and, still, fe...
4999    [Atiku, ', s, campaign, is, coming, wit, Arabi...
Name: context, Length: 5000, dtype: object

In [ ]:
#from itertools import chain
#aug_neg_examples = list(chain.from_iterable(train_data['context']))
#aug_data = pd.DataFrame(
#    {'response': aug_neg_examples, 
#     'context': np.empty(len(aug_neg_examples)),
#     'label': np.repeat('NOT_SARCASM', len(aug_neg_examples))
#     })


In [ ]:
#train_data = train_data.append(aug_data)


In [10]:
!nvidia-smi

Tue Nov 17 18:45:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!pip install tensorflow-gpu >> /dev/null
!pip install --upgrade grpcio >> /dev/null
!pip install tqdm  >> /dev/null
!pip install bert-for-tf2 >> /dev/null

In [12]:
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing

In [13]:
%matplotlib inline
RANDOM_SEED = 9
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [14]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-11-17 18:46:43--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.176, 172.217.2.112, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   190MB/s    in 2.0s    

2020-11-17 18:46:45 (190 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [15]:
os.makedirs("model", exist_ok=True)

In [16]:
!mv uncased_L-12_H-768_A-12/ model

In [17]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [18]:
class SarcasmDetectionData:

  def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=72):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, context, label = ' '.join(row['response']),' '.join(row['context']), row['label']
      tokens = ["[CLS]"]
      tokens.extend(self.tokenizer.tokenize(text))
      tokens.append("[PAD]")
      tokens.extend(self.tokenizer.tokenize(context))
      tokens.append("[SEP]")
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [19]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [20]:
def create_model(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)
  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = keras.layers.Dropout(0.3)(cls_out)
  logits = keras.layers.Dense(units=512, activation="tanh")(cls_out)
  logits = keras.layers.Dropout(0.3)(logits)
  logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

In [ ]:
classes = train_data.label.unique().tolist()

data = SarcasmDetectionData(train_data, test_data, tokenizer, classes, max_seq_len=256)

In [22]:
model = create_model(data.max_seq_len, bert_ckpt_file)

Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f7bee1ab0b8> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [23]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 256)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 256, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               393728    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

In [24]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [65]:
modelCheckpoint = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/best_sarcasm_model14.hdf5', save_best_only = True)

In [66]:
history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_data=(data.test_x, data.test_y),
  batch_size=24,
  shuffle=True,
  epochs=15,
  callbacks=[modelCheckpoint]
)

Epoch 1/15
209/209 [==============================] - 364s 2s/step - loss: 0.5966 - acc: 0.6930 - val_loss: 0.5996 - val_acc: 0.7000
Epoch 2/15
209/209 [==============================] - 364s 2s/step - loss: 0.5303 - acc: 0.7722 - val_loss: 0.5821 - val_acc: 0.7122
Epoch 3/15
209/209 [==============================] - 357s 2s/step - loss: 0.4887 - acc: 0.8200 - val_loss: 0.5860 - val_acc: 0.7100
Epoch 4/15
209/209 [==============================] - 362s 2s/step - loss: 0.4649 - acc: 0.8442 - val_loss: 0.5795 - val_acc: 0.7289
Epoch 5/15
209/209 [==============================] - 357s 2s/step - loss: 0.4363 - acc: 0.8738 - val_loss: 0.5801 - val_acc: 0.7211
Epoch 6/15
209/209 [==============================] - 356s 2s/step - loss: 0.4196 - acc: 0.8910 - val_loss: 0.5982 - val_acc: 0.6989
Epoch 7/15
209/209 [==============================] - 357s 2s/step - loss: 0.4081 - acc: 0.9038 - val_loss: 0.5922 - val_acc: 0.7133
Epoch 8/15
209/209 [==============================] - 356s 2s/step - 

In [26]:
model.load_weights('/content/drive/My Drive/best_sarcasm_model8.hdf5')

In [27]:
y_pred = model.predict(data.test_x).argmax(axis=-1)

In [ ]:
print(classification_report(data.test_y, y_pred, target_names=classes))

In [29]:
rows = [['twitter_' + str(i+1), 'SARCASM' if r == 0 else 'NOT_SARCASM'] for i, r in enumerate(y_pred)]
results = pd.DataFrame(rows, columns=["ID", "RESULT"])

In [30]:
results.head()

,ID,RESULT
0,twitter_1,NOT_SARCASM
1,twitter_2,SARCASM
2,twitter_3,SARCASM
3,twitter_4,NOT_SARCASM
4,twitter_5,SARCASM


In [ ]:
results.to_csv('/content/drive/My Drive/answer.txt', header = False, index = False)